🟦 Introduction

- The tasks include:
- Implementing an inner product 
- Implementing a Hadamard layer
- Building the complete Bernstein–Vazirani circuit
- Running the algorithm on a simulator (BasicSimulator or AerSimulator)

In [52]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit_aer import AerSimulator

## Task 1 — Inner Product

### 🔧 Gate Implementation
For each bit index `i` where `a[i] = 1`, apply a **CNOT** gate:
- **Control** = input qubit `i`
- **Target** = output qubit (the final qubit)

### ⚠️ Important Detail
- In the assignment, `a[0]` is the **most significant bit**.
- In Qiskit, qubit `0` is the **least significant qubit**.
- Therefore, you must **reverse the string `a`** before mapping its bits to qubit indices.

In [53]:
def inner_product(circuit: QuantumCircuit, a: str) -> None:
    """
    Modify `circuit` in-place to implement the inner product modulo 2
    with respect to the binary string `a`.

    - The circuit is assumed to have n+1 qubits, where n = len(a).
    - Qubits 0..n-1 are the input register.
    - Qubit n is the output qubit.
    - For each i with a_i = 1, we add CNOT from the corresponding input qubit
      to the output qubit.
    """
    n = len(a)
    # a0 is the most significant bit, q_(n-1)
    # a_(n-1) is least significant, q0
    a_reversed = a[::-1]

    for i, bit in enumerate(a_reversed):
        if bit == '1':
            # control: input qubit i, target: output qubit n
            circuit.cx(i, n)


### Task 1 Test

In [54]:
a = "01101"
n = len(a)
circuit = QuantumCircuit(n + 1, 0)

inner_product(circuit, a)
print(circuit)


                    
q_0: ──■────────────
       │            
q_1: ──┼────────────
       │            
q_2: ──┼────■───────
       │    │       
q_3: ──┼────┼────■──
       │    │    │  
q_4: ──┼────┼────┼──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_5: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘


## Task 2 — Hadamard Layer
This function applies a single layer of Hadamard gates to every qubit in the circuit.
It modifies the circuit in-place, as required by the assignment.


In [55]:
def hadamards(circuit: QuantumCircuit) -> None:
    """
    Add one Hadamard gate to each qubit of `circuit`, in-place.
    """
    for q in range(circuit.num_qubits):
        circuit.h(q)


### Task 2 Test

In [56]:
circuit = QuantumCircuit(5, 0)
hadamards(circuit)
print(circuit)


     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     └───┘


## Task 3 — Full Bernstein–Vazirani Circuit
This function constructs the full Bernstein–Vazirani algorithm for a secret bitstring a.
Steps:
- Create a circuit with n+1 qubits and n classical bits.
- Apply X to the output qubit to initialize it in ∣1⟩.
- Apply Hadamard gates to all qubits.
- Apply the inner_product oracle.
- Apply a second set of Hadamard gates to the first n qubits.
- Measure the input qubits.
The output measurement should reveal the hidden string a.


In [61]:
def bernstein_vazirani(a: str) -> QuantumCircuit:
    """
    Construct the Bernstein-Vazirani circuit for the secret string `a`.

    Returns a QuantumCircuit with:
      - n+1 qubits  (n input + 1 output)
      - n classical bits (for measuring the input qubits)
    """
    n = len(a)
    circuit = QuantumCircuit(n + 1, n)

    # Step 3: Flip the output qubit |0> -> |1>
    circuit.x(n)
    circuit.barrier()

    # Step 4: Hadamard layer on all qubits (input + output)
    hadamards(circuit)
    circuit.barrier()

    # Step 5: Inner product oracle for secret string a
    inner_product(circuit, a)
    circuit.barrier()

    # Step 6: Another Hadamard layer on the input qubits only
    for q in range(n):
        circuit.h(q)
    circuit.barrier()

    # Step 7: Measure input qubits 0..n-1 into classical bits 0..n-1
    for q in range(n):
        circuit.measure(q, q)

    return circuit


### Test Task 3

In [62]:
a = "01101"
circuit = bernstein_vazirani(a)
print(circuit)


           ░ ┌───┐ ░                 ░ ┌───┐ ░ ┌─┐            
q_0: ──────░─┤ H ├─░───■─────────────░─┤ H ├─░─┤M├────────────
           ░ ├───┤ ░   │             ░ ├───┤ ░ └╥┘┌─┐         
q_1: ──────░─┤ H ├─░───┼─────────────░─┤ H ├─░──╫─┤M├─────────
           ░ ├───┤ ░   │             ░ ├───┤ ░  ║ └╥┘┌─┐      
q_2: ──────░─┤ H ├─░───┼────■────────░─┤ H ├─░──╫──╫─┤M├──────
           ░ ├───┤ ░   │    │        ░ ├───┤ ░  ║  ║ └╥┘┌─┐   
q_3: ──────░─┤ H ├─░───┼────┼────■───░─┤ H ├─░──╫──╫──╫─┤M├───
           ░ ├───┤ ░   │    │    │   ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐
q_4: ──────░─┤ H ├─░───┼────┼────┼───░─┤ H ├─░──╫──╫──╫──╫─┤M├
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║  ║  ║ └╥┘
q_5: ┤ X ├─░─┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────░──╫──╫──╫──╫──╫─
     └───┘ ░ └───┘ ░ └───┘└───┘└───┘ ░       ░  ║  ║  ║  ║  ║ 
c: 5/═══════════════════════════════════════════╩══╩══╩══╩══╩═
                                                0  1  2  3  4 


## Task 4 — Running the Algorithm
This function executes the Bernstein–Vazirani circuit using either:

BasicSimulator()

AerSimulator()

The measurement results should return the hidden string a with probability 1


In [59]:
def run_bernstein_vazirani(a: str, shots: int = 1024, use_aer: bool = False):
    """
    Build and simulate the Bernstein-Vazirani circuit for secret string `a`.
    Prints the counts dictionary and returns it.
    
    If use_aer=True, uses AerSimulator instead of BasicSimulator.
    """
    circuit = bernstein_vazirani(a)

    if use_aer:
        backend = AerSimulator()
    else:
        backend = BasicSimulator()

    # Transpile for the chosen backend
    compiled = transpile(circuit, backend)

    # Run the job
    job = backend.run(compiled, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print(circuit)
    print("Secret string a:", a)
    print("Measurement result counts:", counts)
    return counts


In [60]:
counts = run_bernstein_vazirani("01101", shots=1024, use_aer=False)


           ░ ┌───┐ ░                 ░ ┌───┐ ░ ┌─┐            
q_0: ──────░─┤ H ├─░───■─────────────░─┤ H ├─░─┤M├────────────
           ░ ├───┤ ░   │             ░ ├───┤ ░ └╥┘┌─┐         
q_1: ──────░─┤ H ├─░───┼─────────────░─┤ H ├─░──╫─┤M├─────────
           ░ ├───┤ ░   │             ░ ├───┤ ░  ║ └╥┘┌─┐      
q_2: ──────░─┤ H ├─░───┼────■────────░─┤ H ├─░──╫──╫─┤M├──────
           ░ ├───┤ ░   │    │        ░ ├───┤ ░  ║  ║ └╥┘┌─┐   
q_3: ──────░─┤ H ├─░───┼────┼────■───░─┤ H ├─░──╫──╫──╫─┤M├───
           ░ ├───┤ ░   │    │    │   ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐
q_4: ──────░─┤ H ├─░───┼────┼────┼───░─┤ H ├─░──╫──╫──╫──╫─┤M├
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║  ║  ║ └╥┘
q_5: ┤ X ├─░─┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────░──╫──╫──╫──╫──╫─
     └───┘ ░ └───┘ ░ └───┘└───┘└───┘ ░       ░  ║  ║  ║  ║  ║ 
c: 5/═══════════════════════════════════════════╩══╩══╩══╩══╩═
                                                0  1  2  3  4 
Secret string a: 01101
Measurement result counts: {'011